## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\admin\\OneDrive - kaist.ac.kr\\바탕 화면\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\admin\\OneDrive - kaist.ac.kr\\바탕 화면\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# batch1: 0~40(41)
# batch2: 41~83(43)
# batch3: 64~123(40)

b1 = 0
b2 = 0
b3 = 0

for cell in bat_sel_dict.keys():
    if cell.startswith('b1'):
        b1 += 1
    elif cell.startswith('b2'):
        b2 += 1
    else:
        b3 += 1

print(b1, b2, b3)

# Tensor dimension: (cell, cycle, time, variable)
regression_cycles = [30, 50, 80]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    globals()[f'x_tensor_{reg}'] = torch.zeros(b1+b2+b3, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1
    
    globals()[f'x_tensor_{reg}'].size()
    
num_variables = globals()[f'x_tensor_{reg}'].shape[3]
time_length = globals()[f'x_tensor_{reg}'].shape[2]

41 43 40


torch.Size([124, 30, 120, 5])

torch.Size([124, 50, 120, 5])

torch.Size([124, 80, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

### Dataloading as 2D CNN's method

In [7]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, batches, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        batches = [0] + batches
        self.batches = batches
        self.test_split = test_split
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        # Issue: 각 batch마다 따로 scaling을 해야하나?
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
#         self.all = list(zip(self.xdata_scaled, self.ydata_scaled))
        
        prev = 0
        for i in range(1, len(self.batches)):
            globals()[f'self.b{i}_xdata_scaled'] = self.xdata_scaled[prev:prev + self.batches[i]]
            globals()[f'self.b{i}_ydata_scaled'] = self.ydata_scaled[prev:prev + self.batches[i]]
            
            prev +=self.batches[i]
            
            globals()[f'self.b{i}_train_size'], globals()[f'self.b{i}_val_size'], globals()[f'self.b{i}_test_size'] = self.get_splits_per_batch(self.batches[i])
            
            globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_x_test_scaled'], globals()[f'self.b{i}_y_train_scaled'], globals()[f'self.b{i}_y_test_scaled'] = train_test_split(globals()[f'self.b{i}_xdata_scaled'], 
                                                                                                            globals()[f'self.b{i}_ydata_scaled'], 
                                                                                                            test_size = globals()[f'self.b{i}_test_size'], 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
            globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_x_val_scaled'], globals()[f'self.b{i}_y_train_scaled'], globals()[f'self.b{i}_y_val_scaled'] = train_test_split(globals()[f'self.b{i}_x_train_scaled'], 
                                                                                                            globals()[f'self.b{i}_y_train_scaled'], 
                                                                                                            test_size = globals()[f'self.b{i}_val_size'], 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
            
            print(f'Batch {i}, total {self.batches[i]} cells')
            print('x_train, x_val, x_test:', globals()[f'self.b{i}_x_train_scaled'].shape, globals()[f'self.b{i}_x_val_scaled'].shape, globals()[f'self.b{i}_x_test_scaled'].shape)
            print('y_train, y_val, y_test:', globals()[f'self.b{i}_y_train_scaled'].shape, globals()[f'self.b{i}_y_val_scaled'].shape, globals()[f'self.b{i}_y_test_scaled'].shape)
        
            globals()[f'self.b{i}_train'] = list(zip(globals()[f'self.b{i}_x_train_scaled'], globals()[f'self.b{i}_y_train_scaled']))
            globals()[f'self.b{i}_val'] = list(zip(globals()[f'self.b{i}_x_val_scaled'], globals()[f'self.b{i}_y_val_scaled']))
            globals()[f'self.b{i}_test'] = list(zip(globals()[f'self.b{i}_x_test_scaled'], globals()[f'self.b{i}_y_test_scaled']))

            globals()[f'self.b{i}_train_dataloader'] = DataLoader(globals()[f'self.b{i}_train'], batch_size = self.batch_size, shuffle=True)
            globals()[f'self.b{i}_val_dataloader'] = DataLoader(globals()[f'self.b{i}_val'], batch_size = self.batch_size, shuffle=True)
            globals()[f'self.b{i}_test_dataloader'] = DataLoader(globals()[f'self.b{i}_test'], batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits_per_batch(self, batch_len):
        # test size
        test_size = int(self.test_split*batch_len)
        # val size
        val_size = int(0.2*(batch_len - test_size))
        # train_size
        train_size = batch_len - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        self.train_dataloader = [globals()[f'self.b1_train_dataloader']]
        self.val_dataloader = [globals()[f'self.b1_val_dataloader']]
        if len(self.batches)>2:
            for i in range(2, len(self.batches)):
                self.train_dataloader.append(globals()[f'self.b{i}_train_dataloader'])
                self.val_dataloader.append(globals()[f'self.b{i}_val_dataloader'])
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        self.test_dataloader = [globals()[f'self.b1_test_dataloader']]
        if len(self.batches)>2:
            for i in range(2, len(self.batches)):
                self.test_dataloader.append(globals()[f'self.b{i}_test_dataloader'])
        return self.test_dataloader
    
    def scaler(self):
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [8]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [30, 50, 80]
# regression_cycles = [100]
batches = [b1, b2, b3]

# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    print(reg, 'cycles')
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, batches = batches, test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler()
    for batch in range(len(batches)):
        globals()[f'b{batch+1}_train_dataloader_{reg}'], globals()[f'b{batch+1}_val_dataloader_{reg}'], globals()[f'b{batch+1}_test_dataloader_{reg}'] = globals()[f'reg_train_dataloader_{reg}'][batch], globals()[f'reg_val_dataloader_{reg}'][batch], globals()[f'reg_test_dataloader_{reg}'][batch]

30 cycles
Batch 1, total 41 cells
x_train, x_val, x_test: (27, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([27]) torch.Size([6]) torch.Size([8])
Batch 2, total 43 cells
x_train, x_val, x_test: (28, 5, 120, 30) (7, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([28]) torch.Size([7]) torch.Size([8])
Batch 3, total 40 cells
x_train, x_val, x_test: (26, 5, 120, 30) (6, 5, 120, 30) (8, 5, 120, 30)
y_train, y_val, y_test: torch.Size([26]) torch.Size([6]) torch.Size([8])
50 cycles
Batch 1, total 41 cells
x_train, x_val, x_test: (27, 5, 120, 50) (6, 5, 120, 50) (8, 5, 120, 50)
y_train, y_val, y_test: torch.Size([27]) torch.Size([6]) torch.Size([8])
Batch 2, total 43 cells
x_train, x_val, x_test: (28, 5, 120, 50) (7, 5, 120, 50) (8, 5, 120, 50)
y_train, y_val, y_test: torch.Size([28]) torch.Size([7]) torch.Size([8])
Batch 3, total 40 cells
x_train, x_val, x_test: (26, 5, 120, 50) (6, 5, 120, 50) (8, 5, 120, 50)
y_train, y_val, y_test: torch.Siz

## Define Model Class 

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [11]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [12]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
#         print("Batch size, input_size, timestep, cycles: ", x.size())
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
#         print('out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)', out1.size())
#         print(hn1.size())
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(out1.size(0)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [13]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [14]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                if n==0:
                    EvalPredictions = yhat
                    EvalActuals = actual
                else:
                    EvalPredictions = np.concatenate((EvalPredictions, yhat), axis = 0)
                    EvalActuals = np.concatenate((EvalActuals, actual), axis = 0)

#             EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                    
                if n==0:
                    TestPredictions = yhat
                    TestActuals = actual
                else:
                    TestPredictions = np.concatenate((TestPredictions, yhat), axis = 0)
                    TestActuals = np.concatenate((TestActuals, actual), axis = 0)

#             TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                targets = targets.numpy()
                
                if n==0:
                    predictions = yhat
                    actuals = targets
                else:
                    predictions = np.concatenate((predictions, yhat), axis = 0)
                    actuals = np.concatenate((actuals, targets), axis = 0)

            # evaluation of entire train data
#             predictions, actuals = np.append(predictions), np.append(actuals)
#             print("predictions: ", predictions)
#             print("actuals: ", actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
#             print("Batch ", i, train_rmse, val_rmse, test_rmse)
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)
    
    def train_val_test_plot_rmse(self, num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
        dataloaders = [self.train_dataloader, self.val_dataloader, self.test_dataloader]

        with torch.no_grad():
            model.eval()

            model_rmse = list()
            dset_index = 0
            for dataloader in dataloaders:
                if dset_index ==0:
                    dset = 'train'
                elif dset_index == 1:
                    dset = 'val'
                else:
                    dset = 'test'

                figadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.jpg')

                predictions, actuals = list(), list()
                for i, (test_input, test_target) in enumerate(dataloader):
                    test_input = test_input.float().to(self.device)
                    test_target = test_target.float().to(self.device)
                    test_yhat, test_ta = model(test_input)

                    if self.device != 'cpu':
                        test_yhat = test_yhat.to('cpu')
                        test_target = test_target.to('cpu')

                    test_yhat = test_yhat.detach().tolist()
                    predictions += test_yhat
                    test_target = test_target.tolist()
                    actuals += test_target

                pred_acc, targ_acc = np.array(predictions), np.array(actuals)

                rmse = mean_squared_error(actuals, predictions, squared = False)
                model_rmse.append(rmse)

                plt.figure(figsize=[10,4])
                plt.rcParams["font.size"] = "12"
                plt.plot(pred_acc, label='Prediction')
                plt.plot(range(len(targ_acc)), targ_acc, label = 'Actual')
                plt.xticks(range(0, len(targ_acc) + 1, int(len(targ_acc)/5)))
                plt.title(f"{rnn1}_hid_{hid1}_nfil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}: RMSE={rmse}")
                plt.xlabel('cell')
                plt.ylabel('Lifespan(cycle)')
                plt.legend()
                plt.savefig(figadd)
                plt.close()

                dset_index += 1

        return model_rmse

## Implementation

In [15]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', f'batch_{batch_num}', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

## Each batch

In [16]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [30, 50, 80]
ep_pats = [[3000, 30]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 5]

batch_nums = [0, 2, 1]

import time

for batch_num in batch_nums:
    for [n_ep, patience] in ep_pats:
        for num_cycles in num_cycless:
            print(f"Batch: {batch_num + 1}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
            trdl = globals()[f'b{batch_num+1}_train_dataloader_{num_cycles}']
            vdl = globals()[f'b{batch_num+1}_val_dataloader_{num_cycles}']
            tedl = globals()[f'b{batch_num+1}_test_dataloader_{num_cycles}']
            for rnn1, hid1 in itertools.product(rnns, hids):
                bi1 = False
                if rnn1.startswith('Bi'):
                    bi1 = True
                for pool2, npool2 in itertools.product(pools, npools):
                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
                    for fil2, lr in itertools.product(num_fils, lrs):
                        for nh2 in nh2s:
                            print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")

                            all_hid1 = hid1 * nh2

                            start = time.time()

                            # Construct CNN
                            model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                            # Train
                            model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                            best_model, best_loss, history, ta, ca = model_train.TrainModel()

                            print('best_loss = '+ str(best_loss))

                            history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num+1)

                            # saving best_epoch, loss history, ta score
                            save_data(history, history_add)
                            save_data(best_model.state_dict(), state_dict_add)
                            save_data(ta, ta_add)
                            save_data(ca, ca_add)

                            print("time: ", time.time()-start)
                            print("\n\n")

Batch: 1, num_cycle = 30, epoch = 3000, patience = 30
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  956
best_loss = [319.55032, 197.89891, 303.114]
C:\Users\admin\OneDrive - kaist.ac.kr\바탕 화면\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_30\RNN_TA_CA_1DCNN\LSTM_1D CNN\batch_1\train_history created
C:\Users\admin\OneDrive - kaist.ac.kr\바탕 화면\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_30\RNN_TA_CA_1DCNN\LSTM_1D CNN\batch_1\model created
C:\Users\admin\OneDrive - kaist.ac.kr\바탕 화면\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_30\RNN_TA_CA_1DCNN\LSTM_1D CNN\batch_1\ta created
C:\Users\admin\OneDrive - kaist.ac.kr\바탕 화면\LIB\CNN work\Variable time length\All dataset\30 cycles\Depth Test_col_3000_30\RNN_TA_CA_1DCNN\LSTM_1D CNN\batch_1\ca created
time:  50.80157399177551



rnn =

Ends at final epoch 3000
Best epoch: 3000
best_loss = [645.2107, 473.07883, 591.09]
time:  159.31203198432922



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  118
best_loss = [248.59114, 76.89193, 172.3257]
time:  7.839010000228882



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.01
Ends at final epoch 3000
Best epoch: 3000
best_loss = [501.00955, 300.5637, 438.2798]
time:  159.28499913215637



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  15
best_loss = [373.16522, 105.810745, 330.86755]
time:  2.3622145652770996



rnn = LSTM, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1080
best_loss = [308.37534, 115.70909, 200.93225]
time:  57.50694704055786



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1677
best

Early stopping at best epoch:  164
best_loss = [250.86208, 73.49198, 177.90585]
time:  9.340927124023438



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  29
best_loss = [199.04327, 109.973946, 154.78772]
time:  2.666536569595337



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.01
Ends at final epoch 3000
Best epoch: 3000
best_loss = [500.84982, 300.36044, 438.10806]
time:  146.47376918792725



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  661
best_loss = [395.12817, 250.67653, 323.3571]
time:  32.595741271972656



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  661
best_loss = [356.43176, 190.90205, 284.9069]
time:  32.79273438453674



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  105
best_loss

Early stopping at best epoch:  85
best_loss = [170.04977, 47.38666, 120.17806]
time:  5.893063545227051



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  53
best_loss = [167.8583, 51.65522, 139.50067]
time:  4.378488302230835



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  849
best_loss = [509.72433, 324.18256, 456.6581]
time:  48.96544861793518



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1009
best_loss = [311.0688, 134.2936, 276.9842]
time:  58.250911712646484



rnn = LSTM, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  115
best_loss = [261.8826, 80.59332, 203.04779]
time:  8.407853841781616



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.001


Early stopping at best epoch:  19
best_loss = [386.01715, 65.399734, 273.28543]
time:  2.501232862472534



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1287
best_loss = [337.42468, 110.76737, 229.36127]
time:  66.25905060768127



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  661
best_loss = [295.5332, 101.03209, 222.0566]
time:  36.51074934005737



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  118
best_loss = [308.2883, 61.429676, 219.0685]
time:  7.598201274871826



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  67
best_loss = [258.07422, 123.316475, 184.22404]
time:  5.3242347240448



rnn = LSTM, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  99
best_loss = [194.1759

...

Early stopping at best epoch:  54
best_loss = [59.540974, 22.047012, 99.75872]
time:  4.492021083831787



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  33
best_loss = [62.330833, 24.154081, 86.75973]
time:  3.3676607608795166



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  462
best_loss = [66.95607, 15.568342, 99.32114]
time:  28.46655035018921



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  423
best_loss = [52.412807, 51.025364, 116.8392]
time:  27.484654903411865



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 3, lr = 0.001
Early stopping at best epoch:  49
best_loss = [84.436646, 15.096118, 83.40602]
time:  4.630402326583862



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.001
Ear

Early stopping at best epoch:  20
best_loss = [56.653557, 24.984558, 89.597176]
time:  3.2723875045776367



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  452
best_loss = [54.586903, 23.865479, 105.36639]
time:  32.23669409751892



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  497
best_loss = [54.241215, 23.373028, 93.86738]
time:  36.05085110664368



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.001
Early stopping at best epoch:  49
best_loss = [69.80918, 19.460154, 91.96792]
time:  5.343975305557251



rnn = RNN, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  71
best_loss = [57.19169, 26.09175, 95.02588]
time:  6.866614818572998



rnn = RNN, bi = False, hid per head = 3, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  110
best_loss = [45.67096, 22.1

Early stopping at best epoch:  311
best_loss = [40.462486, 35.341774, 99.84095]
time:  20.4222149848938



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.001
Early stopping at best epoch:  81
best_loss = [80.72133, 32.830906, 88.69701]
time:  6.5473082065582275



rnn = RNN, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  89
best_loss = [38.794666, 16.359581, 98.3098]
time:  6.992682456970215



rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 7, lr = 0.01
Early stopping at best epoch:  11
best_loss = [70.07265, 27.297071, 91.001686]
time:  2.4143922328948975



rnn = RNN, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  32
best_loss = [56.159843, 14.833281, 114.1398]
time:  3.650048017501831



Pooling: 2 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 5, num_heads = 1, num_fil = 3, lr = 0.0001
Ends at 

Early stopping at best epoch:  67
best_loss = [76.61605, 18.244905, 58.789066]
time:  5.214883327484131



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  93
best_loss = [44.23992, 30.11889, 87.926704]
time:  6.898482322692871



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 3, lr = 0.01
Early stopping at best epoch:  94
best_loss = [42.55037, 14.359816, 111.82733]
time:  6.544161796569824



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  29
best_loss = [58.90287, 25.06094, 121.24476]
time:  3.266219139099121



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  433
best_loss = [65.45198, 34.098, 96.59488]
time:  24.875646352767944



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  462
best_loss = [45.24809, 87.34783, 1

Early stopping at best epoch:  53
best_loss = [44.606148, 38.949814, 108.60068]
time:  5.164923906326294



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 5, lr = 0.01
Early stopping at best epoch:  9
best_loss = [61.096493, 26.095123, 94.46806]
time:  2.3181161880493164



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  54
best_loss = [58.002792, 10.023137, 112.18218]
time:  5.100940704345703



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.0001
Ends at final epoch 3000
Best epoch: 3000
best_loss = [293.22598, 293.50388, 314.65582]
time:  183.64316654205322



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  606
best_loss = [45.67912, 52.462166, 136.15158]
time:  39.2833468914032



rnn = RNN, bi = False, hid per head = 7, num_heads = 1, num_fil = 7, lr = 0.001
Ends at final epoch 3000
Best epoch: 3000
best_loss =

In [ ]:
# Hyperparameters
num_vars = globals()[f'x_tensor_{reg}'].size()[3]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycless = [30, 50, 80]
ep_pats = [[1000, 10], [2000, 20]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2, 3, 4, 5]

batch_nums = [0, 2]

import time

for batch_num in batch_nums:
    for [n_ep, patience] in ep_pats:
        for num_cycles in num_cycless:
            print(f"Batch: {batch_num + 1}, num_cycle = {num_cycles}, epoch = {n_ep}, patience = {patience}")
            trdl = globals()[f'b{batch_num+1}_train_dataloader_{num_cycles}']
            vdl = globals()[f'b{batch_num+1}_val_dataloader_{num_cycles}']
            tedl = globals()[f'b{batch_num+1}_test_dataloader_{num_cycles}']
            for rnn1, hid1 in itertools.product(rnns, hids):
                bi1 = False
                if rnn1.startswith('Bi'):
                    bi1 = True
                for pool2, npool2 in itertools.product(pools, npools):
                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
                    for fil2, lr in itertools.product(num_fils, lrs):
                        for nh2 in nh2s:
                            print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")

                            all_hid1 = hid1 * nh2

                            start = time.time()

                            # Construct CNN
                            model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                            # Train
                            model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                            best_model, best_loss, history, ta, ca = model_train.TrainModel()

                            print('best_loss = '+ str(best_loss))

                            history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num+1)

                            # saving best_epoch, loss history, ta score
                            save_data(history, history_add)
                            save_data(best_model.state_dict(), state_dict_add)
                            save_data(ta, ta_add)
                            save_data(ca, ca_add)

                            print("time: ", time.time()-start)
                            print("\n\n")

Batch: 1, num_cycle = 30, epoch = 1000, patience = 10
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  408
best_loss = [609.49207, 431.30588, 559.7886]
time:  16.924480438232422



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  336
best_loss = [618.90594, 450.49362, 563.3058]
time:  14.0671706199646



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  601
best_loss = [455.06058, 267.98193, 390.25705]
time:  25.04930543899536



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.0001
Ends at final epoch 1000
Best epoch: 1000
best_loss = [662.22296, 492.44458, 608.8851]
time:  41.11353039741516



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  92
best_

Early stopping at best epoch:  71
best_loss = [289.79303, 128.65942, 236.46027]
time:  3.4454917907714844



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.001
Early stopping at best epoch:  75
best_loss = [266.14352, 73.82571, 165.4843]
time:  3.628197193145752



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  15
best_loss = [298.4618, 119.95319, 225.14243]
time:  1.0707676410675049



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  10
best_loss = [330.3591, 60.057114, 285.24738]
time:  0.8501546382904053



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  9
best_loss = [339.21283, 120.405975, 220.68346]
time:  0.8136508464813232



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [334.01163, 9

Early stopping at best epoch:  538
best_loss = [597.8853, 427.1858, 539.82855]
time:  23.458171844482422



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  542
best_loss = [499.92972, 312.89444, 440.5174]
time:  23.66606903076172



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  751
best_loss = [586.492, 413.858, 529.1698]
time:  32.8752167224884



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  112
best_loss = [266.2843, 91.359436, 152.70758]
time:  5.255959749221802



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  108
best_loss = [295.25192, 80.22458, 197.8536]
time:  5.085066556930542



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  84
best_loss = [276.61072, 7

Early stopping at best epoch:  8
best_loss = [408.82837, 117.17339, 254.30373]
time:  0.7963366508483887



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.01
Early stopping at best epoch:  12
best_loss = [308.33255, 105.64565, 198.73146]
time:  0.9896974563598633



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.01
Early stopping at best epoch:  16
best_loss = [280.0433, 106.42014, 178.91731]
time:  1.1511504650115967



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  460
best_loss = [531.8241, 360.49753, 474.08823]
time:  21.115912914276123



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  583
best_loss = [502.19864, 325.11166, 437.74924]
time:  26.498461961746216



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  460
best_loss = [56

Early stopping at best epoch:  68
best_loss = [252.6109, 107.73461, 118.22287]
time:  3.1843504905700684



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  124
best_loss = [216.2133, 78.460686, 166.3705]
time:  5.459046125411987



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.001
Early stopping at best epoch:  105
best_loss = [235.93178, 67.206314, 150.89613]
time:  4.773037672042847



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  75
best_loss = [270.15396, 107.4923, 183.0911]
time:  3.5122528076171875



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  9
best_loss = [347.6885, 82.15186, 246.55661]
time:  0.7923502922058105



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  25
best_loss = [198.27425, 11

...

Ends at final epoch 2000
Best epoch: 2000
best_loss = [661.8699, 492.0442, 608.51624]
time:  109.44858598709106



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  53
best_loss = [364.33286, 125.35015, 279.7935]
time:  3.9012317657470703



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  55
best_loss = [297.00433, 65.99001, 252.98242]
time:  4.0248658657073975



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  68
best_loss = [291.46768, 98.01544, 152.06721]
time:  4.809036016464233



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 5, lr = 0.001
Ends at final epoch 2000
Best epoch: 2000
best_loss = [651.3216, 480.05066, 597.48535]
time:  111.32370781898499



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  25
best_los

Early stopping at best epoch:  35
best_loss = [111.81312, 76.11918, 76.27812]
time:  3.242656946182251



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  316
best_loss = [343.72577, 88.824615, 233.51834]
time:  19.1916024684906



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  307
best_loss = [366.39713, 110.499565, 290.04147]
time:  18.33677840232849



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  722
best_loss = [332.74033, 176.53316, 258.8181]
time:  41.772502422332764



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  307
best_loss = [366.4538, 135.35216, 273.54184]
time:  18.57950258255005



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  37
best_loss = [276

Early stopping at best epoch:  89
best_loss = [180.58107, 101.39442, 92.41128]
time:  5.655706405639648



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.001
Early stopping at best epoch:  67
best_loss = [215.91612, 91.839905, 151.85873]
time:  4.763036489486694



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  47
best_loss = [156.7022, 62.098255, 70.493164]
time:  3.3908467292785645



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  50
best_loss = [155.62888, 44.832314, 100.448654]
time:  3.582357168197632



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  29
best_loss = [148.77412, 73.60245, 152.89136]
time:  2.531747341156006



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  50
best_loss = [189.69296, 71

Early stopping at best epoch:  501
best_loss = [348.0006, 236.79497, 307.0048]
time:  27.167609930038452



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  811
best_loss = [370.44537, 170.39647, 289.27417]
time:  43.49621272087097



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  601
best_loss = [349.9975, 183.8233, 283.93198]
time:  34.19969201087952



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  67
best_loss = [344.6885, 262.76215, 319.55142]
time:  4.524832010269165



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  62
best_loss = [334.90134, 120.659676, 287.59964]
time:  4.3668975830078125



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.001
Early stopping at best epoch:  97
best_loss = [275.5

Early stopping at best epoch:  43
best_loss = [137.40474, 70.70727, 149.6379]
time:  3.418112277984619



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  35
best_loss = [116.06234, 70.40868, 58.805847]
time:  3.058828592300415



rnn = LSTM, bi = False, hid per head = 7, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  67
best_loss = [87.41948, 42.927124, 86.6032]
time:  4.80744481086731



rnn = LSTM, bi = False, hid per head = 7, num_heads = 5, num_fil = 3, lr = 0.01
Ends at final epoch 2000
Best epoch: 2000
best_loss = [551.82306, 363.51364, 492.5849]
time:  116.12917876243591



rnn = LSTM, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  338
best_loss = [326.0416, 128.45163, 252.63405]
time:  19.29202699661255



rnn = LSTM, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  661
best_loss = [313.915

In [ ]:
modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [1, 2, 3, 4, 5]

batch_nums = [0, 2, 1]

for batch_num in batch_nums:
    for [n_ep, patience] in ep_pats:
        for nh2 in nh2s:
            for rnn1 in rnns:
                df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
                bi1 = False
                for hid1 in hids:
                    for pool2, npool2 in itertools.product(pools, npools):
                        for fil2, lr in itertools.product(num_fils, lrs):
                            history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                        rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr, batch_num+1)

                            history = load_data(history_add)

                            df.loc[f'Batch {batch_num+1}_{nh2}_{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

                df
                df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, 'Each batch', f'Batch {batch_num+1}'], 
                                filename = f'Batch {batch_num+1}_{nh2} heads_{modelname}_{rnn1}_{nh2}.csv')
                df.to_csv(df_add)